In [1]:
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier
import lightgbm as lgb
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [2]:
#load in test, train and concat the datasets
train = pd.read_csv('train.csv')

In [3]:
pd.options.display.max_columns = 999
train.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,0.59881,1.423200e+09,0.27240,9.455600,-0.050305,1938.300,8.6331,4.0607,26.8670,-1.180,10961.00,1.5397,135.3200,-1.49650,440.080,2.590100e+12,2.194200e+09,2968800.0,0.001431,13.3270,0.75050,18509.0,146820.0,-0.000276,1.090600e+16,1705.400,414.29,3.5392,1888.0,0.968930,18.3880,-0.001583,7.7059,5.9325,0.025693,4.5604,0.61122,10.7950,0.341930,0.235010,NaN,5237.70,1.29610,163.66,0.403780,0.188600,-0.001446,-0.35416,6.6432,0.30534,0.514020,1.907300e+09,29.861,0.965010,1797.2,72.178,108.6200,1.9799,1.2907,0.99519,1.3228,827.340,7.779900e+14,4.129900e+10,0.006994,6.9835,43956.0,1978.2,5.5084,-0.001081,6.1244,1.231800e+11,275.9200,5308500.0,1704.000,5.022400e+10,53.3980,-2.2012,6871.0,3.8862,-0.00558,5252.100,166.690,1.60740,0.66534,7768.900,0.99662,1.125700e+11,2.2432,0.934160,0.65056,94569.0,21.471,8214.100,0.288010,0.097826,0.001071,1.412400e+09,0.11093,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,1.20930,3.334100e+09,0.28631,-0.012858,-0.019912,10.284,6.1872,1.0419,4.6404,31.877,123620.00,1.3951,125.8100,1.19890,136.450,9.098100e+09,4.004100e+10,1564000.0,0.000204,3.1074,1.50330,238000.0,21440.0,-0.001344,3.079400e+16,229.100,844.82,1.4680,4726.5,0.915380,-1.5321,0.982600,7.1112,2.0797,0.042321,4.2523,0.41871,5.4499,0.012737,0.386470,7.3082,283.21,-0.92552,140.80,0.247390,-0.001656,-0.000975,-0.22629,2.4246,0.77147,0.011613,1.803700e+09,64.604,0.262650,4455.0,78.339,745.5100,2.9069,1.4826,1.00510,1.4974,84.446,3.505600e+15,2.242300e+09,0.896300,4.6749,17713.0,9003.1,-4.3546,0.254100,6.9191,1.832400e+11,9.6510,32800.0,1480.600,2.300600e+10,44.0510,205.6900,4295.3,13.3880,0.46843,754.610,83.233,1.18900,29.55000,7343.700,0.99815,4.877700e+13,1.2708,-0.000969,5.29520,6779.0,227.720,34.342,0.340300,0.143370,0.049276,1.903200e+09,0.97673,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,0.38164,1.230300e+09,0.25807,2.455600,NaN,26.873,7.5463,1.9967,1.9526,817.760,-2948.70,2.0054,1.6826,1.19680,74.624,-3.273900e+10,5.718900e+10,11058.0,-0.003097,8.0241,1.13180,27940.0,862460.0,-0.002207,5.849100e+13,-897.840,NaN,1.3561,3063.4,0.086232,16.1060,0.001481,11.4760,5.3430,0.012162,4.1018,-0.88270,8.1228,-0.676690,0.337700,-1.0732,4097.00,13.45800,159.24,0.322300,0.560090,0.000455,-0.16083,3.5753,0.60970,0.028301,5.271300e+08,14.454,0.115490,14605.0,36.992,-9.6391,64.2670,NaN,0.99278,2.5891,430.400,-4.453500e+13,5.144900e+12,0.099591,6.5516,1887.5,43319.0,4.3931,0.260260,6.1052,1.013300e+11,357.2700,1476600.0,90.845,1.306200e+09,2.3731,391.3700,2965.3,NaN,0.49459,43.524,138.520,1.10790,0.91948,47.915,NaN,1.510500e+12,3.4663,0.560950,4.13090,95531.0,39.486,-83.148,0.084881,0.032222,0.001668,1.436500e+07,0.20102,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1
3,3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,1.10960,1.223100e+09,0.30944,10.370000,-0.106260,533.840,7.8490,1.0379,8.0030,12.349,-195.28,2.5598,92.1420,0.63789,1054.900,-1.204100e+10,5.187300e+12,1475400.0,1.036500,1.1903,0.98941,301200.0,NaN,-0.000007,-9.299200e+13,-10.818,1020.30,2.9553,3342.5,-0.000372,17.0110,0.095268,5.7448,15.8830,0.037934,4.4860,-0.88909,8.4384,-1.189800,0.001391,NaN,175.81,67.

In [4]:
feature_cols = [col for col in train.columns if 'f' in col]

In [5]:
#scaling and imputing of missing values
sc = StandardScaler()
si = SimpleImputer()

train[feature_cols] = si.fit_transform(sc.fit_transform(train[feature_cols]))

In [6]:
#calc Z scores for df
z_scores = stats.zscore(train[feature_cols])

In [7]:
#get the number of outliers by column
abs_z_scores = np.abs(z_scores)
outliers = np.count_nonzero(abs_z_scores>3,axis=1)
train['outlier'] = outliers

In [8]:
#drop any records with more than 6 outlier columns
X = train[train['outlier']<6][feature_cols]
y = train[train['outlier']<6]['claim']

In [9]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)

In [10]:
print('X train shape ',len(X_train))
print('y train shape ',len(y_train))
print('X test shape ',len(X_test))
print('y test shape ',len(y_test))

X train shape  765483
y train shape  765483
X test shape  191371
y test shape  191371


In [11]:
#set Stratified K Fold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [12]:
#setup XGBoost model (need GPU to run tree_method = 'gpu hist')
model_xgb =  XGBClassifier(max_depth=3,
                              colsample_bytree=0.7,
                              n_estimators=20000,
                              learning_rate=0.02,
                              objective='binary:logistic', 
                              verbosity =1,
                              eval_metric  = 'auc',
                              tree_method='gpu_hist',
                              n_jobs=-1,
                              use_label_encoder=False)

In [13]:
model_lgb = lgb.LGBMClassifier(objective= 'binary',
    n_estimators= 20000,
    random_state= 42,
    learning_rate= 0.02,
    subsample= 0.6,
    subsample_freq= 1,
    colsample_bytree= 0.4,
    reg_alpha= 10.0,
    reg_lambda= 1e-1,
    min_child_weight= 256,
    min_child_samples= 20,
    max_depth = 3,
    num_leaves = 7,
    eval_metric = 'auc')

In [15]:
for fold_index, (train_index,val_index) in enumerate(folds.split(X_train,y_train)):
    print('Batch {} started...'.format(fold_index))
    gc.collect()
    bst = model_xgb.fit(X_train.iloc[train_index],y_train.iloc[train_index],
              eval_set = [(X_train.iloc[val_index],y_train.iloc[val_index])],
              early_stopping_rounds=200,
              verbose= 200, 
              eval_metric ='auc')
    bst_2 = model_lgb.fit(X_train.iloc[train_index],y_train.iloc[train_index],
              eval_set = [(X_train.iloc[val_index],y_train.iloc[val_index])],
              early_stopping_rounds=200,
              verbose= 200, 
              eval_metric ='auc')

Batch 0 started...
[0]	validation_0-auc:0.52116
[200]	validation_0-auc:0.67179
[400]	validation_0-auc:0.71429
[600]	validation_0-auc:0.73608
[800]	validation_0-auc:0.75015
[1000]	validation_0-auc:0.76123
[1200]	validation_0-auc:0.76946
[1400]	validation_0-auc:0.77559
[1600]	validation_0-auc:0.78062
[1800]	validation_0-auc:0.78480
[2000]	validation_0-auc:0.78797
[2200]	validation_0-auc:0.79013
[2400]	validation_0-auc:0.79174
[2600]	validation_0-auc:0.79292
[2800]	validation_0-auc:0.79390
[3000]	validation_0-auc:0.79471
[3200]	validation_0-auc:0.79544
[3400]	validation_0-auc:0.79594
[3600]	validation_0-auc:0.79632
[3800]	validation_0-auc:0.79668
[4000]	validation_0-auc:0.79703
[4200]	validation_0-auc:0.79733
[4400]	validation_0-auc:0.79755
[4600]	validation_0-auc:0.79782
[4800]	validation_0-auc:0.79799
[5000]	validation_0-auc:0.79819
[5200]	validation_0-auc:0.79834
[5400]	validation_0-auc:0.79849
[5600]	validation_0-auc:0.79865
[5800]	validation_0-auc:0.79877
[6000]	validation_0-auc:0.79

[4000]	valid_0's auc: 0.791748	valid_0's binary_logloss: 0.589047
[4200]	valid_0's auc: 0.792114	valid_0's binary_logloss: 0.587813
[4400]	valid_0's auc: 0.792374	valid_0's binary_logloss: 0.586784
[4600]	valid_0's auc: 0.792649	valid_0's binary_logloss: 0.585782
[4800]	valid_0's auc: 0.792885	valid_0's binary_logloss: 0.584865
[5000]	valid_0's auc: 0.79308	valid_0's binary_logloss: 0.584115
[5200]	valid_0's auc: 0.793304	valid_0's binary_logloss: 0.583335
[5400]	valid_0's auc: 0.793466	valid_0's binary_logloss: 0.582642
[5600]	valid_0's auc: 0.793651	valid_0's binary_logloss: 0.58201
[5800]	valid_0's auc: 0.79381	valid_0's binary_logloss: 0.581454
[6000]	valid_0's auc: 0.793926	valid_0's binary_logloss: 0.580956
[6200]	valid_0's auc: 0.79403	valid_0's binary_logloss: 0.580508
[6400]	valid_0's auc: 0.794097	valid_0's binary_logloss: 0.580104
[6600]	valid_0's auc: 0.794175	valid_0's binary_logloss: 0.579714
[6800]	valid_0's auc: 0.794247	valid_0's binary_logloss: 0.579363
[7000]	valid_0

[2400]	validation_0-auc:0.79143
[2600]	validation_0-auc:0.79261
[2800]	validation_0-auc:0.79355
[3000]	validation_0-auc:0.79430
[3200]	validation_0-auc:0.79486
[3400]	validation_0-auc:0.79534
[3600]	validation_0-auc:0.79583
[3800]	validation_0-auc:0.79611
[4000]	validation_0-auc:0.79634
[4200]	validation_0-auc:0.79651
[4400]	validation_0-auc:0.79675
[4600]	validation_0-auc:0.79701
[4800]	validation_0-auc:0.79718
[5000]	validation_0-auc:0.79734
[5200]	validation_0-auc:0.79749
[5400]	validation_0-auc:0.79757
[5600]	validation_0-auc:0.79765
[5800]	validation_0-auc:0.79771
[6000]	validation_0-auc:0.79782
[6200]	validation_0-auc:0.79790
[6400]	validation_0-auc:0.79799
[6600]	validation_0-auc:0.79803
[6800]	validation_0-auc:0.79804
[7000]	validation_0-auc:0.79811
[7200]	validation_0-auc:0.79817
[7400]	validation_0-auc:0.79824
[7600]	validation_0-auc:0.79829
[7800]	validation_0-auc:0.79833
[8000]	validation_0-auc:0.79835
[8200]	validation_0-auc:0.79838
[8400]	validation_0-auc:0.79841
[8600]	v

[7000]	valid_0's auc: 0.799553	valid_0's binary_logloss: 0.575919
[7200]	valid_0's auc: 0.799623	valid_0's binary_logloss: 0.575596
[7400]	valid_0's auc: 0.799693	valid_0's binary_logloss: 0.575285
[7600]	valid_0's auc: 0.799766	valid_0's binary_logloss: 0.574993
[7800]	valid_0's auc: 0.799843	valid_0's binary_logloss: 0.574726
[8000]	valid_0's auc: 0.79993	valid_0's binary_logloss: 0.574477
[8200]	valid_0's auc: 0.799965	valid_0's binary_logloss: 0.574278
[8400]	valid_0's auc: 0.800014	valid_0's binary_logloss: 0.574087
[8600]	valid_0's auc: 0.800036	valid_0's binary_logloss: 0.573884
[8800]	valid_0's auc: 0.800123	valid_0's binary_logloss: 0.573676
[9000]	valid_0's auc: 0.800125	valid_0's binary_logloss: 0.573524
Early stopping, best iteration is:
[8865]	valid_0's auc: 0.800141	valid_0's binary_logloss: 0.573611
Batch 5 started...
[0]	validation_0-auc:0.51598
[200]	validation_0-auc:0.67118
[400]	validation_0-auc:0.71423
[600]	validation_0-auc:0.73565
[800]	validation_0-auc:0.74829
[1

[1600]	valid_0's auc: 0.779589	valid_0's binary_logloss: 0.619171
[1800]	valid_0's auc: 0.782822	valid_0's binary_logloss: 0.614448
[2000]	valid_0's auc: 0.785101	valid_0's binary_logloss: 0.610475
[2200]	valid_0's auc: 0.787236	valid_0's binary_logloss: 0.606647
[2400]	valid_0's auc: 0.788924	valid_0's binary_logloss: 0.603384
[2600]	valid_0's auc: 0.790039	valid_0's binary_logloss: 0.600586
[2800]	valid_0's auc: 0.791183	valid_0's binary_logloss: 0.597898
[3000]	valid_0's auc: 0.792029	valid_0's binary_logloss: 0.595624
[3200]	valid_0's auc: 0.792843	valid_0's binary_logloss: 0.593394
[3400]	valid_0's auc: 0.793503	valid_0's binary_logloss: 0.591461
[3600]	valid_0's auc: 0.794078	valid_0's binary_logloss: 0.589742
[3800]	valid_0's auc: 0.794568	valid_0's binary_logloss: 0.588194
[4000]	valid_0's auc: 0.794881	valid_0's binary_logloss: 0.586925
[4200]	valid_0's auc: 0.79524	valid_0's binary_logloss: 0.58561
[4400]	valid_0's auc: 0.79556	valid_0's binary_logloss: 0.584462
[4600]	valid_

Batch 8 started...
[0]	validation_0-auc:0.51927
[200]	validation_0-auc:0.66812
[400]	validation_0-auc:0.71177
[600]	validation_0-auc:0.73547
[800]	validation_0-auc:0.74903
[1000]	validation_0-auc:0.75982
[1200]	validation_0-auc:0.76842
[1400]	validation_0-auc:0.77435
[1600]	validation_0-auc:0.77929
[1800]	validation_0-auc:0.78308
[2000]	validation_0-auc:0.78598
[2200]	validation_0-auc:0.78800
[2400]	validation_0-auc:0.78951
[2600]	validation_0-auc:0.79070
[2800]	validation_0-auc:0.79162
[3000]	validation_0-auc:0.79237
[3200]	validation_0-auc:0.79292
[3400]	validation_0-auc:0.79344
[3600]	validation_0-auc:0.79383
[3800]	validation_0-auc:0.79428
[4000]	validation_0-auc:0.79468
[4200]	validation_0-auc:0.79497
[4400]	validation_0-auc:0.79524
[4600]	validation_0-auc:0.79551
[4800]	validation_0-auc:0.79570
[5000]	validation_0-auc:0.79589
[5200]	validation_0-auc:0.79604
[5400]	validation_0-auc:0.79618
[5600]	validation_0-auc:0.79627
[5800]	validation_0-auc:0.79639
[6000]	validation_0-auc:0.79

[5400]	valid_0's auc: 0.797155	valid_0's binary_logloss: 0.580566
[5600]	valid_0's auc: 0.797296	valid_0's binary_logloss: 0.579927
[5800]	valid_0's auc: 0.797381	valid_0's binary_logloss: 0.579426
[6000]	valid_0's auc: 0.797505	valid_0's binary_logloss: 0.578875
[6200]	valid_0's auc: 0.797639	valid_0's binary_logloss: 0.578361
[6400]	valid_0's auc: 0.79773	valid_0's binary_logloss: 0.577874
[6600]	valid_0's auc: 0.797839	valid_0's binary_logloss: 0.577461
[6800]	valid_0's auc: 0.797952	valid_0's binary_logloss: 0.57704
[7000]	valid_0's auc: 0.798089	valid_0's binary_logloss: 0.576614
[7200]	valid_0's auc: 0.798137	valid_0's binary_logloss: 0.576289
[7400]	valid_0's auc: 0.798175	valid_0's binary_logloss: 0.576002
[7600]	valid_0's auc: 0.798191	valid_0's binary_logloss: 0.57576
[7800]	valid_0's auc: 0.798241	valid_0's binary_logloss: 0.575515
[8000]	valid_0's auc: 0.798259	valid_0's binary_logloss: 0.575288
[8200]	valid_0's auc: 0.798321	valid_0's binary_logloss: 0.575052
[8400]	valid_

In [34]:
model_lgb.booster_.save_model('lgb.txt')

In [35]:
model_xgb.save_model('xgb.json')

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [57]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print(torch.cuda.get_device_name(0))
print()

Using device: cuda
GeForce RTX 2060 SUPER



In [58]:
#convert training date to tensors
X_nn = torch.FloatTensor(np.array(X_train))
y_nn = torch.FloatTensor(np.array(y_train))

In [19]:
batch_size = 1024

In [59]:
#model definition
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X_nn.size(1), 250)
        self.fc2 = nn.Linear(250, 150)
        self.fc3 = nn.Linear(150, 30)
        self.fc4 = nn.Linear(30, 1)
        self.dropout = nn.Dropout(0.15)
        self.bn1 = nn.BatchNorm1d(250)
        self.bn2 = nn.BatchNorm1d(150)
        self.bn3 = nn.BatchNorm1d(30)
        


    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        
      
        return x
net = Net().to(device)
print(net)

Net(
  (fc1): Linear(in_features=118, out_features=250, bias=True)
  (fc2): Linear(in_features=250, out_features=150, bias=True)
  (fc3): Linear(in_features=150, out_features=30, bias=True)
  (fc4): Linear(in_features=30, out_features=1, bias=True)
  (dropout): Dropout(p=0.15, inplace=False)
  (bn1): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [60]:
#reshape into tensor sequences for loading
seq = []
for i in range(0,len(X_nn)):
    seq.append((X_nn[i],y_nn[i]))
    i = i + 1
    

In [61]:
#create trainloader
trainloader = torch.utils.data.DataLoader(seq, batch_size=batch_size,shuffle=True, num_workers=2)

In [62]:
#set loss function and optimizer
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=0.001)
steps = 30
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)
criterion = nn.BCEWithLogitsLoss()

In [63]:
device

device(type='cuda')

In [64]:
from tqdm.notebook import tqdm
net.train().cuda()
net.load_state_dict(torch.load('nn.pth'))

#model training
max_acc = 66.5
min_loss = 1
running_loss = 1
running_val_loss = 1
import time
start_time = time.time()
for epoch in tqdm(range(500)):  # loop over the dataset multiple times
    running_loss = 0.0
    running_val_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, preds = data
        # forward + backward + optimize
        outputs = net(inputs.to(device))
        loss = criterion(outputs, preds.to(device).reshape(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
    print(running_loss/(i + 1))
print('Finished Training')
print("time",(time.time()-start_time))

0.6427841575387965
0.6425306463942808
0.6422869893796941
0.6422884226801561
0.6426763679573243
0.6421121614661446
0.6422560356039414
0.6422819947813921
0.6422955220077127
0.6415933634348732
0.6419891914263128
0.6421304154682924
0.6420572977015041
0.641772390686892
0.6416320574474844
0.6414608428663111
0.6419932916362018
0.6413873504189884
0.6416101664464104
0.6417883289529678
0.6420730563567921
0.6414661457672476
0.641509046050954
0.6413614917247691
0.6412351642222328
0.6412284891395008
0.6410008250551428
0.6408863276402581
0.6411242142399365
0.6409949837999548
0.6411924998071742
0.6413379165737386
0.6409047410927992
0.6406813608133857
0.6409062252325171
0.6408582864438787
0.6408453840304186
0.6403545255170149
0.6407980304510198
0.6404540518866503
0.6406765111627426
0.6404620139675344
0.6405891445390681
0.6399952281605114
0.640118390879529
0.6403011780849752
0.6400126533552924
0.6405396684605807
0.6400522549840856
0.64021248310645
0.6404015754155297
0.6399094207401581
0.640376996866521

KeyboardInterrupt: 

In [65]:
torch.save(net.state_dict(), 'nn.pth')

In [26]:
#convert training date to tensors
X_nn_t = torch.FloatTensor(np.array(X_test))
y_nn_t = torch.FloatTensor(np.array(y_test))

In [66]:
net.eval().cpu()
roc_auc_score(y_test,torch.sigmoid(net(X_nn_t)).detach().numpy().flatten())

0.5308725364736363

In [38]:
roc_auc_score(y_test,model_xgb.predict_proba(X_test)[:,1])

0.7974488009129919

In [39]:
roc_auc_score(y_test,model_lgb.predict_proba(X_test)[:,1])

0.7976420156248671

In [47]:
torch.sigmoid(net(X_nn_t))

tensor([[0.6359],
        [0.6268],
        [0.5177],
        ...,
        [0.6399],
        [0.7442],
        [0.4977]], grad_fn=<SigmoidBackward>)